In [1]:
from fastapi import FastAPI, HTTPException
from bson.objectid import ObjectId
from pymongo import mongo_client
from pymongo.errors import PyMongoError

In [2]:
import requests
import os
import json

In [3]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.relpath("./")))
secret_file = os.path.join(BASE_DIR, '../fastapi-server/app/secret.json')

with open(secret_file) as f:
    secrets = json.loads(f.read())

def get_secret(setting, secrets=secrets):
    try:
        return secrets[setting]
    except KeyError:
        errorMsg = "Set the {} environment variable.".format(setting)
        return errorMsg

HOSTNAME = 'localhost:27017'
USERNAME = get_secret("ATLAS_Username")
PASSWORD = get_secret("ATLAS_Password")
SEOUL_API_KEY = get_secret("SEOUL_API_KEY")

In [17]:
USERNAME = "root"
PASSWORD = "1234"
HOSTNAME = "localhost:27017"  # 포트 번호가 포함된 호스트 이름

In [4]:
location = '강남역'

In [5]:
url = f"http://openapi.seoul.go.kr:8088/{SEOUL_API_KEY}/json/citydata/1/2/{location}"
response = requests.get(url)
data = response.json()
city_data = data['CITYDATA']

In [9]:
city_data['LIVE_PPLTN_STTS'][0]['FCST_PPLTN']

[{'FCST_TIME': '2024-05-01 11:00',
  'FCST_CONGEST_LVL': '여유',
  'FCST_PPLTN_MIN': '38000',
  'FCST_PPLTN_MAX': '40000'},
 {'FCST_TIME': '2024-05-01 12:00',
  'FCST_CONGEST_LVL': '보통',
  'FCST_PPLTN_MIN': '46000',
  'FCST_PPLTN_MAX': '48000'},
 {'FCST_TIME': '2024-05-01 13:00',
  'FCST_CONGEST_LVL': '보통',
  'FCST_PPLTN_MIN': '52000',
  'FCST_PPLTN_MAX': '54000'},
 {'FCST_TIME': '2024-05-01 14:00',
  'FCST_CONGEST_LVL': '보통',
  'FCST_PPLTN_MIN': '58000',
  'FCST_PPLTN_MAX': '60000'},
 {'FCST_TIME': '2024-05-01 15:00',
  'FCST_CONGEST_LVL': '약간 붐빔',
  'FCST_PPLTN_MIN': '58000',
  'FCST_PPLTN_MAX': '60000'},
 {'FCST_TIME': '2024-05-01 16:00',
  'FCST_CONGEST_LVL': '보통',
  'FCST_PPLTN_MIN': '56000',
  'FCST_PPLTN_MAX': '58000'},
 {'FCST_TIME': '2024-05-01 17:00',
  'FCST_CONGEST_LVL': '보통',
  'FCST_PPLTN_MIN': '56000',
  'FCST_PPLTN_MAX': '58000'},
 {'FCST_TIME': '2024-05-01 18:00',
  'FCST_CONGEST_LVL': '보통',
  'FCST_PPLTN_MIN': '54000',
  'FCST_PPLTN_MAX': '56000'},
 {'FCST_TIME': '2024-

In [19]:
client = mongo_client.MongoClient(f'mongodb://{USERNAME}:{PASSWORD}@{HOSTNAME}')


In [20]:
mydb = client['cg']
mycol = mydb['congestion']

In [22]:
result = mycol.insert_one(city_data)

In [38]:
from datetime import datetime, timedelta


In [40]:
input_time = datetime.now()
one_hour_ago = datetime.now() - timedelta(hours=1)
date_str = input_time.strftime("%Y-%m-%d")
hour = one_hour_ago.hour  # 시간 부분 추출

In [26]:
len(date_str)

10

In [41]:
query = {
    "AREA_NM": '고속터미널역',
    "$expr": {
        "$and": [
            {"$eq": [{"$substr": [{"$arrayElemAt": ["$LIVE_PPLTN_STTS.PPLTN_TIME", 0]}, 0, 10]}, date_str]}, # 첫 번째 요소의 날짜가 일치하는지 확인
            {"$eq": [{"$hour": {"$dateFromString": {"dateString": {"$arrayElemAt": ["$LIVE_PPLTN_STTS.PPLTN_TIME", 0]}}}}, hour]}  # 계산된 1시간 전의 시간
        ]
    }
}

In [42]:
closest_location = '고속터미널역'

In [43]:
data = mydb['congestion'].find_one(query, sort=[('_id', -1)])